selecionar o clock para externo e configurar o awg para 10 MHz

selecionar dual channel with markers

selecionar sampling rate -> 64 GSamples/s

colocar o canal 2 para usar o memória externa -> dividir a memória para dois canais (isso divide o sampling rate)

habilitar todos os canais


# Setting TCP/IP Connection with Sockets

In [1]:
from socket import socket, AF_INET, SOCK_STREAM
from time import sleep
import numpy as np
import matplotlib.pyplot as plt

In [2]:
session = socket(AF_INET, SOCK_STREAM)

In [3]:
session.connect(('169.254.101.100',5025))

In [4]:
session.settimeout(4)

In [5]:

def write(cmd):
    session.sendall(str.encode(cmd+'\n'))

def query(cmd):
    write(cmd)
    sleep(0.1)
    return session.recv(4096).decode()

def getError():
    return query('SYSTEM:ERR?')


In [8]:
query('*IDN?')

'Keysight Technologies,M8195A,MY55A00568,4.1.0.0-2\n'

In [12]:
getError()

'0,"No error"\n'

# Setting AWG

### Clock

In [10]:
write(':rosc:sour ext')

In [11]:
write(':rosc:freq 10e6')

### Sampling Rate

In [13]:
write(':FREQ:RAST 64e9')

### Dual Channel With Marker

In [14]:
write(':inst:dacm dcm')

### Divide Memory (Sampling Rate also)

In [15]:
write(':INST:MEM:EXT:RDIV DIV2')

### Set Channel 2 to External Memory

In [16]:
write(':TRAC2:MMOD EXT')

### Enable all Channels

In [17]:
write(':outp1 on')
write(':outp2 on')
write(':outp3 on')
write(':outp4 on')

### Set offset Voltage and Amplitude Voltage

In [18]:
write(':volt1 1')
write(':volt2 1')
write(':volt3 1')
write(':volt4 1')

write(':volt3:offs 0.5')
write(':volt4:offs 0.5')

# Loading data to AWG

In [19]:
def start():
    write(':INIT:IMM')

def stop():
    write(':abor')

def load(channel,data, offset = 0 ):
    tamanho = len(data)
    n = int(np.log10(tamanho)+1)
    tag = '#'+str(n) + str(tamanho)
    
    cmd = ':trac{}:data 1, {}, '.format(channel,offset)+tag
    
    payload = cmd.encode() + bytes(data) + '\n'.encode()
    
    session.sendall(payload)


### para baixar os dados

In [20]:
write(':trac2:block? 1,0, 4096')

In [21]:
session.recv(4096)

b'#10\n'

### Gerar os dados para canal 2

In [33]:
length = 10e-9
x = np.arange(0,length,1/32e9)

sigma = 0.2 * length
y = 127*np.exp(-(x-length/2)**2/sigma**2)

padding = np.zeros(192)
y = np.append(y,padding)

data2 = np.array(y,dtype=np.int8)

### Gerar os dados para canal 1 com markers

In [51]:
length = 10e-9
x = np.arange(0,length,1/32e9)

sigma = 0.2 * length
y = 127*np.exp(-(x-length/2)**2/sigma**2)

padding = np.zeros(192)
y = np.append(y,padding)

markers = 2*np.ones(len(y))

yWMarkers =  np.array(list(zip(y,markers))).flatten()

data1 = np.array(yWMarkers,dtype=np.int8)

### carregar os dados no AWG

In [34]:
load(2,data2,0) # colocar o dado no canal 1 no offset 0

In [ ]:
load(1,data2,512) # colocar o dado no canal 1 no offset 512

In [35]:
getError()

'0,"No error"\n'

### free memory

In [29]:
write('trac1:del:all')

### alocar memória

In [31]:
write('trac1:def 1,4096,0')

# Para fazer o mesmo mas utilizando os drivers e funções de ajuda que eu criei

In [42]:
from instruments.M8195A_driver import * # awg

In [44]:
awg = M8195A_driver('169.254.101.100') # internamente vai criar a session de socket na porta 5025

In [ ]:
awg.enableChanneloutput(1) # write(':outp1 on')

In [ ]:
awg.setRefInClockFrequency(10e6) # write(':rosc:freq 10e6')

In [ ]:
awg.setRefInClockExternal() # write(':rosc:sour ext')

In [ ]:
# write(':trac2:block? 1,0, 4096')
# session.recv(4096)
# garante que vai baixar todos os dados a partir do offset 0, de tamanho 1024, do channel 1
awg.downloadWaveform(1,0,1024) 

In [49]:
awg.freeMemory() # write('trac1:del:all')

In [ ]:
awg.allocMemory(nbytes) # write('trac1:def 1,{},0'.format(nbytes))

In [ ]:
awg.setChannelMemoryToExtended(channel)  #write(':TRAC{}:MMOD EXT'.format(channel))
awg.setChannelMemoryToInternal(channel)  #write(':TRAC{}:MMOD INT'.format(channel))

awg.getChannelMemorySetting(channel)  #query(':TRAC{}:MMOD?'.format(channel))

awg.setSampleRate(64e9) #write(':FREQ:RAST 64e9')
awg.getSampleRate() #query(':FREQ:RAST?')


awg.setMemoryDivision(2) #write(':INST:MEM:EXT:RDIV DIV2')


In [ ]:
awg.setDualWithMarker() # write(':inst:dacm dcm')

In [ ]:
awg.setVoltage(1,0.5) # write(':volt1 0.5')

awg.setVoltageOffset(1,0.2) # write(':volt1:offs 0.2')


In [ ]:
awg.loadData(data1,0,512) # load(1,data1,512)